In [38]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.5 MB/s eta 0:00:00 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [streamlit]/6 [streamlit]


In [27]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from transformers import pipeline
from bertopic import BERTopic
from sklearn.metrics import accuracy_score, classification_report

In [7]:
df=pd.read_csv("zomato20k.csv")
df.head()

,userName,review,rating,date
0,A Google user,comes on time,5,2025-10-03 13:47:17
1,A Google user,it's amazing,5,2025-10-03 13:46:50
2,A Google user,nice,5,2025-10-03 13:46:23
3,A Google user,correct time delivery... food taste all hotels...,5,2025-10-03 13:46:00
4,A Google user,"i mean ,, its good , when your hunger is on pe...",4,2025-10-03 13:44:39


In [8]:
df["rating"].value_counts()

rating
5    138139
4     28743
1     25177
3      5749
2      2192
Name: count, dtype: int64

In [9]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [10]:
df["review"]=df["review"].apply(preprocess_text)

In [12]:
x=df["review"]
y=df["rating"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [13]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [15]:
train_sentiments=sentiment_pipeline(x_train.tolist(),truncation=True,max_length=512)
predicted_labels = [result['label'] for result in train_sentiments]

In [18]:
df_results = pd.DataFrame({
    'review': x_train,
    'predicted_sentiment': predicted_labels
})
df_results.head()

,review,predicted_sentiment
21743,good,positive
124554,eat repeat,neutral
10351,wait half hour,neutral
135164,goodzomato well adopting,positive
49969,app good,positive


In [19]:
test_sentiments=sentiment_pipeline(x_test.tolist(),truncation=True,max_length=512)
x_test_predicted_labels = [result['label'] for result in test_sentiments]

/home/arpit/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [25]:
def rating_to_sentiment(rating):
    if rating in [4, 5]:
        return 'positive'  
    elif rating == 3:
        return 'neutral'
    else:  
        return 'negative'


true_labels = y_test.apply(rating_to_sentiment)

accuracy = accuracy_score(true_labels, x_test_predicted_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


print("\n Detailed Classification Report:")
print(classification_report(true_labels, x_test_predicted_labels))

Test Accuracy: 83.45%

 Detailed Classification Report:
              precision    recall  f1-score   support

    negative       0.82      0.72      0.76      6855
     neutral       0.07      0.28      0.11      1501
    positive       0.97      0.87      0.92     41644

    accuracy                           0.83     50000
   macro avg       0.62      0.63      0.60     50000
weighted avg       0.92      0.83      0.87     50000



In [23]:
labels=y_train.apply(rating_to_sentiment)
accuracy = accuracy_score(labels, predicted_labels)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

Training Accuracy: 84.05%


In [30]:
topic_model=BERTopic(language="english",verbose=True)

In [32]:
topics, probs = topic_model.fit_transform(x_train)

2025-10-04 15:52:21,099 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████████████████████████████████████████████████████████████████| 4688/4688 [01:49<00:00, 42.84it/s]
2025-10-04 15:54:46,402 - BERTopic - Embedding - Completed ✓
2025-10-04 15:54:46,404 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-04 16:19:51,778 - BERTopic - Dimensionality - Completed ✓
2025-10-04 16:19:51,785 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-04 16:20:05,706 - BERTopic - Cluster - Completed ✓
2025-10-04 16:20:05,765 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-04 16:20:07,393 - BERTopic - Representation - Completed ✓


In [33]:
print(topic_model.get_topic_info().head(6))

   Topic  Count                                               Name  \
0     -1  24477                              -1_dont_order_chat_se   
1      0  29705                             0_good_hahahaha_wbu_wa   
2      1   8927                           1_nice_wandarfool_ii_500   
3      2   3292  2_excellent_excellentcan_excellently_excellentbut   
4      3   3276                            3_doen_beat_exactly_ads   
5      4   1986                     4_service_good_server_provided   

                                      Representation  \
0  [dont, order, chat, se, even, customer, money,...   
1  [good, hahahaha, wbu, wa, progressive, iwant, ...   
2  [nice, wandarfool, ii, 500, ice, god, till, re...   
3  [excellent, excellentcan, excellently, excelle...   
4        [doen, beat, exactly, ads, man, na, , , , ]   
5  [service, good, server, provided, today, , , ,...   

                                 Representative_Docs  
0  [deliver food also getting back delivery partn...  
1     

In [34]:
print(topic_model.get_topic(0))

[('good', 0.00897669230383399), ('hahahaha', 0.00020877410957867662), ('wbu', 0.00020877410957867662), ('wa', 0.00020877410957867662), ('progressive', 0.00018551864955634385), ('iwant', 0.00018551864955634385), ('hehe', 0.00014882225610532443), ('ig', 0.00013927650923691557), ('mee', 0.00011451112911428677), ('lol', 0.00010766326131230235)]


In [36]:

df_test_results = pd.DataFrame({
    'review': x_test,
    'true_rating': y_test,
    'predicted_sentiment': x_test_predicted_labels # from your previous step
})

df_test_results.to_csv('test_results.csv', index=False)

In [41]:
# In your Jupyter Notebook
topic_info_df = topic_model.get_topic_info()
topic_info_df.to_csv('topics.csv', index=False)

In [42]:
!pip list

Package                      Version
---------------------------- --------------------
absl-py                      2.3.1
aiohappyeyeballs             2.6.1
aiohttp                      3.12.15
aiosignal                    1.4.0
altair                       5.5.0
annotated-types              0.7.0
anyio                        4.9.0
argon2-cffi                  25.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    3.0.0
astunparse                   1.6.3
async-lru                    2.0.5
async-timeout                4.0.3
attrs                        25.3.0
babel                        2.17.0
beautifulsoup4               4.13.4
bertopic                     0.17.3
bleach                       6.2.0
blinker                      1.9.0
blis                         1.3.0
Brotli                       1.1.0
cached-property              1.5.2
cachetools                   5.5.2
catalogue                    2.0.10
certifi                      